Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from yeast8model import Yeast8Model

Initialise model

In [ ]:
glc_exch_rate = 16.89
wt_ec = Yeast8Model("./models/ecYeastGEM_batch_8-6-0.xml")
wt_ec.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt_ec.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)

Ablate and store fluxes in each round

In [ ]:
wt_ec.ablation_result = wt_ec.ablate()

ablation_fluxes = wt_ec.ablation_fluxes
ablation_fluxes_diff = ablation_fluxes.copy()
ablation_fluxes_diff.pop("original")
for biomass_component, fluxes in ablation_fluxes_diff.items():
    ablation_fluxes_diff[biomass_component] = (
        ablation_fluxes[biomass_component] - ablation_fluxes["original"]
    )
    print(f"{biomass_component}")
    print(f"min {1e5 * ablation_fluxes_diff[biomass_component].min()} * 1e-5")
    print(f"max {1e5 * ablation_fluxes_diff[biomass_component].max()} * 1e-5")

(Other work -- rename sections later)

In [ ]:
# Load subsystems.csv as a lookup table, convert to dict for speed
subsystem_df = pd.read_csv('easy_subsystems.csv', index_col=0)
subsystem_dict = dict(zip(
    subsystem_df.reaction_id.to_list(),
    subsystem_df.subsystem.to_list()
))

In [ ]:
# Choose component
biomass_component = 'lipid'
s = ablation_fluxes_diff[biomass_component].copy()

# Get top (False) or bottom (True) N
n = 100
s = s.sort_values(ascending=False)[:n]

In [ ]:
# Extract protein IDs
rxn_idx_list = s.index.to_list()
enz_metabolite_ids = [rxn_idx.replace("draw_", "") + "[c]" for rxn_idx in rxn_idx_list]

# Get participating reaction IDs
participating_rxn_ids = []
for enz_metabolite_id in enz_metabolite_ids:
    enz_participating_rxns = list(wt_ec.model.metabolites.get_by_id(enz_metabolite_id)._reaction)
    enz_participating_rxn_ids = [enz_participating_rxn.id for enz_participating_rxn in enz_participating_rxns]
    participating_rxn_ids.extend(enz_participating_rxn_ids)
# unique
participating_rxn_ids = list(set(participating_rxn_ids))

# Subsystems that correspond to the participating reactions
subsystem_list = [
    subsystem_dict[rxn_id[:6]]
    for rxn_id in participating_rxn_ids
    if rxn_id[:2] == "r_"
]

# Frequency table & plot
subsystem_freqs = pd.Series(subsystem_list).value_counts()
ax = subsystem_freqs.plot.barh()
ax.invert_yaxis()
ax.set_xlabel('Number of appearances of subsystem')